In [54]:
from fastai.vision.all import *
from torch.utils.data import Dataset
import torchaudio
import shutil
import os

import optuna
from optuna.trial import *

In [55]:
# Read metadata and split file

# name of the original metadata file
metadata_file = "esc50.csv"

# chosen categories
categories = [
    "airplane",
    "thunderstorm",
    "vacuum_cleaner",
    "cat",
    "chainsaw",
    "dog",
    "chirping_birds",
    "keyboard_typing",
    "fireworks",
    "church_bells",
]

# read metadata, add backslash to filename and filter categories
metadata = pd.read_csv(metadata_file)
metadata["filename"] = "\\" + metadata["filename"]
metadata = metadata[metadata["category"].isin(categories)].reset_index(drop=True)

# assign new target number to each category
for category in categories:
    metadata.loc[metadata["category"] == category, "target"] = categories.index(
        category
    )

# take the 1+ folds as training data and the first fold as validation and test data
metadata_train = metadata[metadata["fold"] > 1].reset_index(drop=True)
metadata_val_test = metadata[metadata["fold"] == 1].reset_index(drop=True)

# list of metadata for validation and test data
metadata_validation = []
metadata_test = []

# sample one item from each category for validation and test data
# repeat until for each category there are no more items left
for category in categories:
    for _ in range(
        round(metadata_val_test[metadata_val_test["category"] == category].shape[0] / 2)
    ):
        item = metadata_val_test[metadata_val_test["category"] == category].sample(2)
        metadata_validation.append(item.iloc[0])
        metadata_test.append(item.iloc[1])
        metadata_val_test = metadata_val_test.drop(item.index)

# convert validation and test metadata to dataframe
metadata_validation = pd.DataFrame(metadata_validation)
metadata_test = pd.DataFrame(metadata_test)

# create category map where each target number has its corresponding category name
category_map = (
    metadata_validation[["target", "category"]].drop_duplicates().reset_index(drop=True)
)

# save metadata and category map to csv
fast_ai_data_folder = "metadata_fastai"
metadata_train.to_csv(
    os.path.join(fast_ai_data_folder, "metadata_train_fastai.csv"), index=False
)
metadata_validation.to_csv(
    os.path.join(fast_ai_data_folder, "metadata_validation_fastai.csv"), index=False
)
metadata_test.to_csv(
    os.path.join(fast_ai_data_folder, "metadata_test_fastai.csv"), index=False
)
category_map.to_csv(
    os.path.join(fast_ai_data_folder, "category_map_fastai.csv"), index=False
)

In [56]:
def time_shift(aud):
    """
    Shifts the signal to the left or right by some percent. Values at the end
    are 'wrapped around' to the start of the transformed signal.
    Args:
        aud: Tuple of the form (audio tensor, sample_rate)
    Returns:
        Tuple: (audio tensor, sample_rate)
    """
    sig, sr = aud
    shift_limit = 0.4

    _, sig_len = sig.shape
    shift_amt = int(random.random() * shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)


def spectro_gram(aud):
    """
    Generates a spectrogram from an audio.
    Args:
        aud: Tuple of the form (audio tensor, sample_rate)
    Returns:
    spec: Pytorch tensor containing the spectrogram
    """
    sig, sr = aud
    top_db = 80
    n_mels = 64
    n_fft = 1024

    # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
    spec = torchaudio.transforms.MelSpectrogram(sr, n_fft=n_fft, n_mels=n_mels)(sig)

    # Convert to decibels
    spec = torchaudio.transforms.AmplitudeToDB(top_db=top_db)(spec)
    return spec


def spectro_augment(spec):
    """
    Augments a spectrogram by masking out some sections of it in both the frequency
    dimension (ie. horizontal bars) and the time dimension (vertical bars) to prevent
    overfitting and to help the model generalise better. The masked sections are
    replaced with the mean value.
    Args:
        spec: Pytorch tensor containing the spectrogram
    Returns:
        Pytorch tensor containing the augmented spectrogram
    """
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec
    max_mask_pct = 0.1
    n_freq_masks = 2
    n_time_masks = 2

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
        aug_spec = torchaudio.transforms.FrequencyMasking(freq_mask_param)(
            aug_spec, mask_value
        )

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
        aug_spec = torchaudio.transforms.TimeMasking(time_mask_param)(
            aug_spec, mask_value
        )

    return aug_spec

In [57]:
def create_spectrogram_tensor(audio, augmentation):
    """
    Create spectrogram tensor from audio tensor.
    Args:
        audio: audio tensor
        augmentation: boolean, whether to apply augmentation
    Returns:
        spectrogram tensor
    """
    if augmentation:
        audio = time_shift(audio)
        spec = spectro_gram(audio)
        spec = spectro_augment(spec)
    else:
        spec = spectro_gram(audio)

    return spec

In [58]:
def create_spectrogram_image(
    metadata,
    category_map,
    data_path_audio,
    data_path_spectrograms,
    augmentation=False,
    part=None,
):
    """
    Create spectrogram image from audio file and save it.
    Args:
        metadata: metadata dataframe
        category_map: category map dataframe
        data_path_audio: path to audio files
        data_path_spectrograms: path to spectrogram images
        augmentation: boolean, whether to apply augmentation
        part: string, part of the dataset
    """
    if not os.path.exists(data_path_spectrograms):
        os.makedirs(data_path_spectrograms)

    for _, content in metadata[["filename", "target"]].iterrows():
        filename = content[0]
        target = content[1]
        category = category_map[target]

        data_path_spectrograms_category = os.path.join(data_path_spectrograms, category)
        if not os.path.exists(data_path_spectrograms_category):
            os.makedirs(data_path_spectrograms_category)

        filename = filename.replace("\\", "")
        audio_file = os.path.join(data_path_audio, filename)
        print(f"Processing {filename}")
        audio = torchaudio.load(audio_file)
        spectrogram_tensor = create_spectrogram_tensor(audio, augmentation)
        spectrogram = spectrogram_tensor.squeeze(0).numpy()

        if part is not None:
            filename = filename.replace(".wav", f"_{part}.png")
        else:
            filename = filename.replace(".wav", ".png")

        plt.imsave(os.path.join(data_path_spectrograms_category, filename), spectrogram)
        print(f"Saved {filename}")

In [59]:
data_path_audio = "audio"
data_path_spectrograms = "spectrograms"

# load metadata and category map
fast_ai_data_folder = "metadata_fastai"
metadata_train = pd.read_csv(
    os.path.join(fast_ai_data_folder, "metadata_train_fastai.csv")
)
metadata_validation = pd.read_csv(
    os.path.join(fast_ai_data_folder, "metadata_validation_fastai.csv")
)
metadata_test = pd.read_csv(
    os.path.join(fast_ai_data_folder, "metadata_test_fastai.csv")
)

category_map = (
    pd.read_csv(os.path.join(fast_ai_data_folder, "category_map_fastai.csv"))
    .set_index("target")
    .to_dict()["category"]
)

# delete spectrograms folder if it exists
shutil.rmtree(data_path_spectrograms)

# create spectrograms for training, validation and test data
data_path_spectrograms_train = os.path.join("spectrograms", "train")
create_spectrogram_image(
    metadata_train,
    category_map,
    data_path_audio,
    data_path_spectrograms_train,
    augmentation=True,
    part=1,
)
create_spectrogram_image(
    metadata_train,
    category_map,
    data_path_audio,
    data_path_spectrograms_train,
    augmentation=True,
    part=2,
)

data_path_spectrograms_validation = os.path.join("spectrograms", "validation")
create_spectrogram_image(
    metadata_validation,
    category_map,
    data_path_audio,
    data_path_spectrograms_validation,
)

data_path_spectrograms_test = os.path.join("spectrograms", "test")
create_spectrogram_image(
    metadata_test, category_map, data_path_audio, data_path_spectrograms_test
)

Processing 2-104105-A-19.wav
Saved 2-104105-A-19_1.png
Processing 2-104105-B-19.wav
Saved 2-104105-B-19_1.png
Processing 2-104168-A-32.wav
Saved 2-104168-A-32_1.png
Processing 2-105270-A-47.wav
Saved 2-105270-A-47_1.png
Processing 2-106072-A-36.wav
Saved 2-106072-A-36_1.png
Processing 2-106073-A-36.wav
Saved 2-106073-A-36_1.png
Processing 2-106849-A-47.wav
Saved 2-106849-A-47_1.png
Processing 2-108760-A-14.wav
Saved 2-108760-A-14_1.png
Processing 2-108760-B-14.wav
Saved 2-108760-B-14_1.png
Processing 2-108761-A-14.wav
Saved 2-108761-A-14_1.png
Processing 2-109316-A-32.wav
Saved 2-109316-A-32_1.png
Processing 2-110010-A-5.wav
Saved 2-110010-A-5_1.png
Processing 2-110011-A-5.wav
Saved 2-110011-A-5_1.png
Processing 2-114280-A-0.wav
Saved 2-114280-A-0_1.png
Processing 2-114587-A-0.wav
Saved 2-114587-A-0_1.png
Processing 2-116400-A-0.wav
Saved 2-116400-A-0_1.png
Processing 2-117271-A-0.wav
Saved 2-117271-A-0_1.png
Processing 2-117615-A-48.wav
Saved 2-117615-A-48_1.png
Processing 2-117615-B-

In [60]:
# create data block
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=GrandparentSplitter(train_name="train", valid_name="validation"),
)

In [61]:
def objective(trial):
    # hyperparameters to be tuned by Optuna.
    architecture = trial.suggest_categorical(
        "architecture", ["resnet18", "resnet34", "resnet50"]
    )
    bs = trial.suggest_categorical("bs", [8, 16, 32, 64, 128])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    freeze_epochs = trial.suggest_int("freeze_epochs", 1, 5)

    # create data loaders
    data_path_spectrograms = "spectrograms"
    dls = dblock.dataloaders(data_path_spectrograms, bs=bs)

    cbs = [EarlyStoppingCallback(patience=2), SaveModelCallback(monitor="accuracy", fname="best_resnet")]

    # create learner
    learn = vision_learner(dls=dls, arch=eval(architecture), metrics=accuracy, path='checkpoints')

    # train model
    learn.fine_tune(
        epochs=10, # maximum number of epochs after unfreezing all layers
        base_lr=lr,
        freeze_epochs=freeze_epochs,
        cbs=cbs,
    )

    # get max accuracy from training
    accuracies = learn.recorder.values
    max_trial_acc = max(accuracies, key=lambda x: x[2])[2]
    
    print(f"Max trial accuracy: {max_trial_acc}")
    
    # save model if it has the highest accuracy so far
    global max_accuracy
    if max_trial_acc > max_accuracy:
        max_accuracy = max_trial_acc
        os.remove('checkpoints/best_resnet.pth')
        os.rename('checkpoints/models/best_resnet.pth', 'checkpoints/best_resnet.pth')

    return max_trial_acc

In [62]:
# initialize best validation accuracy
max_accuracy = 0

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, show_progress_bar=True)

# remove the models folder created by fastai
shutil.rmtree('checkpoints/models')

[I 2023-12-12 19:59:02,510] A new study created in memory with name: no-name-c8d07763-86ed-4376-a8f4-16d5c219224a


  0%|          | 0/30 [00:00<?, ?it/s]

c:\Users\mkarc\AppData\Local\Programs\Python\Python311\Lib\site-packages\fastai\torch_core.py:263: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  return getattr(torch, 'has_mps', False)


epoch,train_loss,valid_loss,accuracy,time
0,3.766638,2.625383,0.125000,00:20
1,3.712928,2.688857,0.100000,00:12
2,3.669974,2.548377,0.150000,00:12


Better model found at epoch 0 with accuracy value: 0.125.
Better model found at epoch 2 with accuracy value: 0.15000000596046448.


...

Better model found at epoch 0 with accuracy value: 0.800000011920929.
Better model found at epoch 1 with accuracy value: 0.8999999761581421.
No improvement since epoch 1: early stopping
Max trial accuracy: 0.8999999761581421
[I 2023-12-12 23:10:36,310] Trial 28 finished with value: 0.8999999761581421 and parameters: {'architecture': 'resnet34', 'bs': 16, 'lr': 0.0028071105300802417, 'freeze_epochs': 3}. Best is trial 13 with value: 0.949999988079071.


epoch,train_loss,valid_loss,accuracy,time
0,2.571043,1.248210,0.625000,00:42
1,1.396322,0.708713,0.775000,00:44


Better model found at epoch 0 with accuracy value: 0.625.
Better model found at epoch 1 with accuracy value: 0.7749999761581421.


epoch,train_loss,valid_loss,accuracy,time
0,0.163218,0.662893,0.775000,01:00
1,0.123309,0.688980,0.825000,00:58
2,0.108024,0.886472,0.750000,01:00


Better model found at epoch 0 with accuracy value: 0.7749999761581421.
Better model found at epoch 1 with accuracy value: 0.824999988079071.
No improvement since epoch 0: early stopping
Max trial accuracy: 0.824999988079071
[I 2023-12-12 23:15:03,937] Trial 29 finished with value: 0.824999988079071 and parameters: {'architecture': 'resnet50', 'bs': 64, 'lr': 0.004104869157570899, 'freeze_epochs': 2}. Best is trial 13 with value: 0.949999988079071.


In [63]:
# print best trial and its parameters
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value:  0.949999988079071
  Params: 
    architecture: resnet34
    bs: 8
    lr: 0.01069194003403398
    freeze_epochs: 2
